In [14]:
import pandas as pd
import json
from datasets import load_dataset, load_from_disk, DatasetDict
import os

- get data:
    - `./download_data.sh --task all`

In [15]:
CACHE_DIR='.cache'

### 1. parsing

In [16]:
train_langs=[
    "UD_Armenian-ArmTDP", "UD_Norwegian-Nynorsk", "UD_Portuguese-Bosque", "UD_Italian-PoSTWITA", "UD_Old_French-SRCMF", "UD_North_Sami-Giella", "UD_Norwegian-Bokmaal", "UD_French-ParisStories", "UD_Italian-MarkIT", "UD_Chinese-GSDSimp", "UD_English-EWT", "UD_French-Rhapsodie", "UD_French-ParTUT", "UD_Classical_Chinese-Kyoto", "UD_Norwegian-NynorskLIA", "UD_Arabic-NYUAD", "UD_Portuguese-PetroGold", "UD_Italian-TWITTIRO", "UD_Turkish_German-SAGT", "UD_Maghrebi_Arabic_French-Arabizi", "UD_Portuguese-CINTIL", "UD_Ligurian-GLT", "UD_Dutch-Alpino", "UD_Western_Armenian-ArmTDP", "UD_Portuguese-GSD", "singlish", "UD_Arabic-PADT"
]

In [17]:
print(train_langs)

['UD_Armenian-ArmTDP', 'UD_Norwegian-Nynorsk', 'UD_Portuguese-Bosque', 'UD_Italian-PoSTWITA', 'UD_Old_French-SRCMF', 'UD_North_Sami-Giella', 'UD_Norwegian-Bokmaal', 'UD_French-ParisStories', 'UD_Italian-MarkIT', 'UD_Chinese-GSDSimp', 'UD_English-EWT', 'UD_French-Rhapsodie', 'UD_French-ParTUT', 'UD_Classical_Chinese-Kyoto', 'UD_Norwegian-NynorskLIA', 'UD_Arabic-NYUAD', 'UD_Portuguese-PetroGold', 'UD_Italian-TWITTIRO', 'UD_Turkish_German-SAGT', 'UD_Maghrebi_Arabic_French-Arabizi', 'UD_Portuguese-CINTIL', 'UD_Ligurian-GLT', 'UD_Dutch-Alpino', 'UD_Western_Armenian-ArmTDP', 'UD_Portuguese-GSD', 'singlish', 'UD_Arabic-PADT']


In [18]:
f = open('metadata/udp_metadata.json')
metadata = json.load(f)
# Closing file
f.close()

- we finetune each lang where train split available
- if train split not available, we perform zero-shot from UD_English-EWT

In [19]:
all_lang=pd.DataFrame(metadata).T
all_lang.reset_index(level=0, inplace=True)
# all_lang['lang-code'] = list(all_lang.index)
all_lang.head(10)

,index,lang,code,desc,langgroup,split
0,singlish,singlish,eng-sing,,English,"[dev, test, train]"
1,UD_Armenian-ArmTDP,Armenian,hye-east,,Armenian,"[train, test, dev]"
2,UD_French-ParTUT,French,fre-multigenre,,French,"[train, dev, test]"
3,UD_English-EWT,English,eng,,English,"[dev, test, train]"
4,UD_Ligurian-GLT,Ligurian,lij,,Italian,"[train, test]"
5,UD_Gheg-GPS,Gheg,aln,,Albanian,[test]
6,UD_Norwegian-Nynorsk,Norwegian,nor-nynorsk,,Norwegian,"[test, dev, train]"
7,UD_Albanian-TSA,Albanian,alb,,Albanian,[test]
8,UD_Italian-PUD,Italian,ita-trans,,Italian,[test]
9,UD_Portuguese-Bosque,Portuguese,por-euro-bra,,Portuguese,"[test, dev, train]"


In [22]:
#explore a single lang
lang='UD_Armenian-ArmTDP'
dataset = load_dataset("scripts/universal_dependencies.py", lang,
            cache_dir=CACHE_DIR)

Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_Armenian-ArmTDP/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 131.30it/s]


In [25]:
dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [28]:
counts={
    'train':0,'validation':0,'test':0
}

for lang in all_lang['index']:
    dataset = load_dataset("scripts/universal_dependencies.py", lang,
            cache_dir=CACHE_DIR)
    for key in dataset.keys():
        counts[key]+=len(dataset[key])

Reusing dataset universal_dependencies (.cache/universal_dependencies/singlish/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 742.00it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_Armenian-ArmTDP/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 789.74it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_French-ParTUT/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 575.75it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_French-Rhapsodie/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 478.64it/s]


Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 380.78it/s]


Dataset universal_dependencies downloaded and prepared to .cache/universal_dependencies/UD_Mbya_Guarani-Thomas/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 497.78it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_Classical_Chinese-Kyoto/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 330.98it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_Norwegian-NynorskLIA/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 540.16it/s]


...

Dataset universal_dependencies downloaded and prepared to .cache/universal_dependencies/UD_Arabic-PUD/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 582.46it/s]


Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 682.56it/s]


Dataset universal_dependencies downloaded and prepared to .cache/universal_dependencies/UD_Komi_Zyrian-IKDP/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 530.52it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_Western_Armenian-ArmTDP/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 481.55it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_Portuguese-GSD/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 529.65it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_Arab

In [30]:
print(len(all_lang),counts)

51 {'train': 251435, 'validation': 31440, 'test': 44324}


In [9]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 1974
    })
    validation: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 249
    })
    test: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 277
    })
})


- parsing use tokens and deprel column

In [10]:
print(dataset['train']['tokens'][:5])
print()
print(dataset['train']['deprel'][:5])

[['Մտածում', 'եմ', '՝', 'Ադամի', 'ու', 'Եվայի', 'վտարումը', 'Եդեմական', 'այգուց', '(', 'դրախտից', ')', ',', 'նրանց', 'տեղափոխումն', 'այլ', 'վայր', ',', 'ուր', 'շրջակայքն', 'այլևս', 'բարեկամական', 'չէր', ',', 'այլ', 'խիստ', 'թշնամական', ',', 'ուր', 'իրենց', 'հացը', 'պիտի', 'տանջանքով', 'վաստակեին', ',', 'նույն', 'մոլորակի', 'սահմաններում', 'չէր', ',', 'որ', 'կատարվեց', ':'], ['Եդեմն', 'այլ', 'մոլորակ', 'էր', ',', 'աքսորավայրը', '՝', 'այլ', ',', 'այսինքն', '՝', 'այս', ',', 'ուր', 'այժմ', 'էլ', 'բնակվում', 'ենք', ',', 'բայց', 'միշտ', 'նայում', 'ենք', 'երկինք', '՝', 'բնազդում', 'դրոշմված', 'հիշողությամբ', 'Եդեմը', 'որոնելով', '։'], ['Իսկ', 'այն', 'չկա', ',', 'Տերը', 'պայթեցրել', 'է', 'կամ', 'գուցե', 'ամայացրել', ',', 'բնակության', 'համար', 'անպիտան', 'դարձրել', ',', 'կամ', 'էլ', 'կա', 'ու', 'ախտահանվում', 'է', '՝', 'նոր', 'բնակիչներ', 'ընդունելու', 'համար', '։'], ['Մի', 'խոսքով', '՝', 'մենք', 'դրա', 'հետ', 'էլ', 'գործ', 'չունենք', ',', 'մերը', 'չէ', 'այլևս', ',', 'մերը', 'սա', 'է', '՝', 'դ

### 2. pos tagging

In [31]:
train_langs=["UD_Armenian-ArmTDP", "UD_Norwegian-Nynorsk", "UD_Portuguese-Bosque", "UD_Italian-PoSTWITA", "UD_Old_French-SRCMF", "UD_North_Sami-Giella", "UD_Norwegian-Bokmaal", "UD_French-ParisStories", "UD_Italian-MarkIT", "UD_Chinese-GSDSimp", "UD_English-EWT", "UD_French-Rhapsodie", "UD_French-ParTUT", "UD_Classical_Chinese-Kyoto", "UD_Norwegian-NynorskLIA", "UD_Arabic-NYUAD", "UD_Portuguese-PetroGold", "UD_Italian-TWITTIRO", "UD_Turkish_German-SAGT", "UD_Maghrebi_Arabic_French-Arabizi", "UD_Portuguese-CINTIL", "UD_Ligurian-GLT", "UD_Dutch-Alpino", "UD_Western_Armenian-ArmTDP", "UD_Portuguese-GSD", "singlish", "UD_Arabic-PADT", "UD_French-GSD", "UD_Catalan-AnCora", "UD_Estonian-EDT", "UD_Finnish-TDT", "UD_Spanish-AnCora"]

In [32]:
print(train_langs)

['UD_Armenian-ArmTDP', 'UD_Norwegian-Nynorsk', 'UD_Portuguese-Bosque', 'UD_Italian-PoSTWITA', 'UD_Old_French-SRCMF', 'UD_North_Sami-Giella', 'UD_Norwegian-Bokmaal', 'UD_French-ParisStories', 'UD_Italian-MarkIT', 'UD_Chinese-GSDSimp', 'UD_English-EWT', 'UD_French-Rhapsodie', 'UD_French-ParTUT', 'UD_Classical_Chinese-Kyoto', 'UD_Norwegian-NynorskLIA', 'UD_Arabic-NYUAD', 'UD_Portuguese-PetroGold', 'UD_Italian-TWITTIRO', 'UD_Turkish_German-SAGT', 'UD_Maghrebi_Arabic_French-Arabizi', 'UD_Portuguese-CINTIL', 'UD_Ligurian-GLT', 'UD_Dutch-Alpino', 'UD_Western_Armenian-ArmTDP', 'UD_Portuguese-GSD', 'singlish', 'UD_Arabic-PADT', 'UD_French-GSD', 'UD_Catalan-AnCora', 'UD_Estonian-EDT', 'UD_Finnish-TDT', 'UD_Spanish-AnCora']


- we finetune each lang where train split available
- if train split not available, we perform zero-shot from UD_English-EWT

In [34]:
f = open('metadata/pos_metadata.json')
metadata = json.load(f)
# Closing file
f.close()

In [35]:
all_lang=pd.DataFrame(metadata).T
all_lang.reset_index(level=0, inplace=True)
all_lang.head(10)

,index,lang,code,desc,langgroup,split,dataset
0,singlish,singlish,eng-sing,,English,"[dev, test, train]",ud
1,ROci,occitan,ROci,,French-occ,[test],noisy
2,UD_Armenian-ArmTDP,Armenian,hye-east,,Armenian,"[train, test, dev]",ud
3,UD_French-ParTUT,French,fre-multigenre,,French,"[train, dev, test]",ud
4,UD_English-EWT,English,eng,,English,"[dev, test, train]",ud
5,UD_Ligurian-GLT,Ligurian,lij,,Italian,"[train, test]",ud
6,UD_Gheg-GPS,Gheg,aln,,Albanian,[test],ud
7,UD_Norwegian-Nynorsk,Norwegian,nor-nynorsk,,Norwegian,"[test, dev, train]",ud
8,UD_Albanian-TSA,Albanian,alb,,Albanian,[test],ud
9,UD_Italian-PUD,Italian,ita-trans,,Italian,[test],ud


#### use different dataset loading script depending on the dataset

In [36]:
lang='singlish' ## metadata[lang]['dataset']=='ud':
script="scripts/universal_dependencies.py"
predict_dataset = load_dataset(script, lang, cache_dir=CACHE_DIR)
print(predict_dataset)
print(predict_dataset['test']['tokens'][:2])
print(predict_dataset['test']['upos'][:2])

Reusing dataset universal_dependencies (.cache/universal_dependencies/singlish/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 569.70it/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 2465
    })
    validation: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 286
    })
    test: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 299
    })
})
[['bt', 'still', 'okie', 'la', 'if', 'go', 'hong', 'kong', '.', '.'], ['Semb', 'Corp', '3.28', 'coming', '.']]
[[10, 14, 6, 15, 5, 16, 10, 10, 1, 1], [10, 10, 3, 16, 1]]


In [43]:
counts={
    'train':0,'validation':0,'test':0
}
script="scripts/universal_dependencies.py"
ud_ones=all_lang[all_lang['dataset']=='ud']
for lang in ud_ones['index']:
    dataset = load_dataset(script, lang,
            cache_dir=CACHE_DIR)
    for key in dataset.keys():
        counts[key]+=len(dataset[key])

Reusing dataset universal_dependencies (.cache/universal_dependencies/singlish/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 558.77it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_Armenian-ArmTDP/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 760.43it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_French-ParTUT/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 455.95it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_French-Rhapsodie/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 541.81it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_Mbya_Guarani-Thomas/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 362.83it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_Classi

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 375.97it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_Komi_Zyrian-IKDP/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 396.96it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_Western_Armenian-ArmTDP/2.7.0/965e769bf67498e9a458ccb7524974ab67c0b8e377537a4a33c2d025c8fc4fe6)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 500.10it/s]
Reusing dataset universal_dependencies (.cache/universal_dependencies/UD_Po

ValueError: BuilderConfig UD_Spanish-AnCora not found. Available: ['UD_Armenian-ArmTDP', 'UD_Gheg-GPS', 'UD_Norwegian-Nynorsk', 'UD_Albanian-TSA', 'UD_Italian-PUD', 'UD_Portuguese-Bosque', 'UD_Italian-PoSTWITA', 'UD_Old_French-SRCMF', 'UD_Swiss_German-UZH', 'UD_North_Sami-Giella', 'UD_Norwegian-Bokmaal', 'UD_French-ParisStories', 'UD_German-LIT', 'UD_Italian-MarkIT', 'UD_Chinese-GSDSimp', 'UD_Chinese-HK', 'UD_Umbrian-IKUVINA', 'UD_Guarani-OldTuDeT', 'UD_Low_Saxon-LSDC', 'UD_French-Rhapsodie', 'UD_Mbya_Guarani-Thomas', 'UD_French-ParTUT', 'UD_Classical_Chinese-Kyoto', 'UD_Norwegian-NynorskLIA', 'UD_Komi_Permyak-UH', 'UD_Chinese-CFL', 'UD_Arabic-NYUAD', 'UD_Portuguese-PUD', 'UD_Portuguese-PetroGold', 'UD_Italian-TWITTIRO', 'UD_Neapolitan-RB', 'UD_Turkish_German-SAGT', 'UD_Chinese-PUD', 'UD_Maghrebi_Arabic_French-Arabizi', 'UD_Portuguese-CINTIL', 'UD_French-PUD', 'UD_South_Levantine_Arabic-MADAR', 'UD_Komi_Zyrian-Lattice', 'UD_Frisian_Dutch-Fame', 'UD_Skolt_Sami-Giellagas', 'UD_Mbya_Guara

In [ ]:
counts

In [40]:
lang='ROci' ## metadata[lang]['dataset']=='noisy':
script="scripts/pos_tagging/noisy_dialect.py"
data_dir="data/pos_tagging"
predict_dataset = load_dataset(script, lang, data_dir=data_dir,cache_dir=CACHE_DIR)
print(predict_dataset)
print(predict_dataset['test']['tokens'][:2])
print(predict_dataset['test']['upos'][:2])

Using custom data configuration ROci-data_dir=data%2Fpos_tagging


Dataset noisy_dialect downloaded and prepared to .cache/noisy_dialect/ROci-data_dir=data%2Fpos_tagging/1.1.0/a0e4190d7e72716271d4c2496d5dcf596262a4082468db5160758db6f295efe5. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 828.42it/s]

DatasetDict({
    test: Dataset({
        features: ['id', 'tokens', 'upos'],
        num_rows: 874
    })
})
[['Puei', ',', 'sabètz', ',', 'se', 'fins_a', 'cinc', 'ans', 'avètz', 'gaire', 'de', 'sovenirs', ',', 'après', 'un', 'pichòt', 'saup', 'chifrar', 'e', 'pòu', 'jutjar', 'de', 'son', 'sicut', '.'], ['Me', 'rèstan', 'pasmens', 'quauquei', 'sovenirs', "d'", 'avans', 'cinc', 'ans', '.']]
[[14, 1, 16, 1, 5, 2, 3, 0, 16, 14, 2, 0, 1, 2, 8, 0, 16, 16, 9, 16, 16, 2, 8, 0, 1], [11, 16, 14, 11, 0, 2, 2, 3, 0, 1]]


In [44]:
script="scripts/pos_tagging/noisy_dialect.py"
ud_ones=all_lang[all_lang['dataset']=='noisy']
for lang in ud_ones['index']:
    dataset = load_dataset(script, lang, data_dir=data_dir,
            cache_dir=CACHE_DIR)
    for key in dataset.keys():
        counts[key]+=len(dataset[key])

Using custom data configuration ROci-data_dir=data%2Fpos_tagging
Reusing dataset noisy_dialect (.cache/noisy_dialect/ROci-data_dir=data%2Fpos_tagging/1.1.0/a0e4190d7e72716271d4c2496d5dcf596262a4082468db5160758db6f295efe5)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 479.57it/s]
Using custom data configuration dar-egy-data_dir=data%2Fpos_tagging
Reusing dataset noisy_dialect (.cache/noisy_dialect/dar-egy-data_dir=data%2Fpos_tagging/1.1.0/a0e4190d7e72716271d4c2496d5dcf596262a4082468db5160758db6f295efe5)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 635.40it/s]
Using custom data configuration dar-glf-data_dir=data%2Fpos_tagging
Reusing dataset noisy_dialect (.cache/noisy_dialect/dar-glf-data_dir=data%2Fpos_taggin

In [47]:
print(len(all_lang),counts)

66 {'train': 251435, 'validation': 31440, 'test': 100604}


### 3. ner

- we train one language/dialect per language/dialect group and evaluate on the whole group

In [48]:
wikiann_train_langs=["ar", "az", "ku", "tr", "hsb", "nl", "fr", "zh", "en", "mhr", "it", "de", "pa", "es", "hr", "lv", "hi", "ro", "el", "bn"]

In [49]:
print(wikiann_train_langs)

['ar', 'az', 'ku', 'tr', 'hsb', 'nl', 'fr', 'zh', 'en', 'mhr', 'it', 'de', 'pa', 'es', 'hr', 'lv', 'hi', 'ro', 'el', 'bn']


In [50]:
norwegian_train_langs=["bokmaal" ,"nynorsk" ,"samnorsk"]

In [51]:
print(norwegian_train_langs)

['bokmaal', 'nynorsk', 'samnorsk']


In [52]:
f = open('metadata/ner_metadata.json')
metadata = json.load(f)
# Closing file
f.close()

- langgroup corresponds to the training language per group

In [56]:
all_lang=pd.DataFrame(metadata).T
all_lang

,lang,code,langgroup,huggingface,dataset,region,train_lang
ar,arabic,ar,arabic,True,wikiann,arabic,ar
arz,egyptian arabic,arz,arabic,True,wikiann,arabic,ar
kab,kabyle,kab,arabic,False,wikiann,arabic,ar
kbd,kabardian,kbd,adyghe,False,wikiann,adyghe,en
ady,adyghe,ady,adyghe,False,wikiann,adyghe,en
...,...,...,...,...,...,...,...
as,assamese,as,bangla,True,wikiann,bangla,bn
bn,bangla,bn,bangla,True,wikiann,bangla,bn
nob,bokmaal,nob,norwegian,True,norwegian_ner,norwegian,bokmaal
nno,nynorsk,nno,norwegian,True,norwegian_ner,norwegian,nynorsk


- different dataset loading script for different datasets

In [60]:
counts={
    'train':0,'validation':0,'test':0
}
script="wikiann"
ud_ones=all_lang[(all_lang['huggingface']==True) & (all_lang['dataset']=='wikiann')]
for lang in ud_ones['code']:
    dataset = load_dataset(script, lang,
            cache_dir=CACHE_DIR)
    for key in dataset.keys():
        counts[key]+=len(dataset[key])

Reusing dataset wikiann (.cache/wikiann/ar/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 701.94it/s]
Reusing dataset wikiann (.cache/wikiann/arz/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 909.04it/s]
Reusing dataset wikiann (.cache/wikiann/az/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 876.19it/s]
Reusing dataset wikiann (.cache/wikiann/ckb/1.1.0/4bfd4fe446

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 745.70it/s]
Reusing dataset wikiann (.cache/wikiann/mhr/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 902.52it/s]
Reusing dataset wikiann (.cache/wikiann/lij/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 741.87it/s]
Reusing dataset wikiann (.cache/wikiann/nap/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
100%|█████████████████████████████████████████████████████

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 738.35it/s]
Reusing dataset wikiann (.cache/wikiann/oc/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 975.27it/s]
Reusing dataset wikiann (.cache/wikiann/pms/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1039.22it/s]
Reusing dataset wikiann (.cache/wikiann/lv/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
100%|███████████████████████████████████████████████████████

In [61]:
script="scripts/ner/wikiann_og.py"
ud_ones=all_lang[(all_lang['huggingface']==False) & (all_lang['dataset']=='wikiann')]
for lang in ud_ones['code']:
    dataset = load_dataset(script, lang,
            cache_dir=CACHE_DIR)
    for key in dataset.keys():
        counts[key]+=len(dataset[key])

Dataset wikiann_og downloaded and prepared to .cache/wikiann_og/kab/1.1.0/bd5069ae42633af8e332aa82fb4f866eafd1dffc876e6bae94c18978db74e5d7. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 834.69it/s]


Dataset wikiann_og downloaded and prepared to .cache/wikiann_og/kbd/1.1.0/bd5069ae42633af8e332aa82fb4f866eafd1dffc876e6bae94c18978db74e5d7. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 883.38it/s]


Dataset wikiann_og downloaded and prepared to .cache/wikiann_og/ady/1.1.0/bd5069ae42633af8e332aa82fb4f866eafd1dffc876e6bae94c18978db74e5d7. Subsequent calls will reuse this data.


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 864.45it/s]


Dataset wikiann_og downloaded and prepared to .cache/wikiann_og/pnt/1.1.0/bd5069ae42633af8e332aa82fb4f866eafd1dffc876e6bae94c18978db74e5d7. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 797.55it/s]

Dataset wikiann_og downloaded and prepared to .cache/wikiann_og/roa-rup/1.1.0/bd5069ae42633af8e332aa82fb4f866eafd1dffc876e6bae94c18978db74e5d7. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 865.70it/s]


In [63]:
ud_ones

,lang,code,langgroup,huggingface,dataset,region,train_lang
nob,bokmaal,nob,norwegian,True,norwegian_ner,norwegian,bokmaal
nno,nynorsk,nno,norwegian,True,norwegian_ner,norwegian,nynorsk
samnorsk,samnorsk,samnorsk,norwegian,True,norwegian_ner,norwegian,samnorsk


In [64]:
script="scripts/ner/norwegian_ner.py"
ud_ones=all_lang[(all_lang['huggingface']==True) & (all_lang['dataset']=='norwegian_ner')]
for lang in ud_ones['lang']:
    dataset = load_dataset(script, lang,
            cache_dir=CACHE_DIR)
    for key in dataset.keys():
        counts[key]+=len(dataset[key])

Extracting data files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1202.96it/s]


Dataset norwegian_ner downloaded and prepared to .cache/norwegian_ner/bokmaal/1.0.0/d326c5669fdd52e99c78eaa6c9c8a8758a48f7494bf59ed17e12814eb349e443. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 805.82it/s]


Extracting data files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1348.94it/s]


Dataset norwegian_ner downloaded and prepared to .cache/norwegian_ner/nynorsk/1.0.0/d326c5669fdd52e99c78eaa6c9c8a8758a48f7494bf59ed17e12814eb349e443. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 827.82it/s]


Extracting data files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1490.87it/s]


Dataset norwegian_ner downloaded and prepared to .cache/norwegian_ner/samnorsk/1.0.0/d326c5669fdd52e99c78eaa6c9c8a8758a48f7494bf59ed17e12814eb349e443. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 687.59it/s]


In [66]:
print(counts, len(all_lang))

{'train': 459640, 'validation': 190200, 'test': 226841} 89


In [23]:
## for langs not present in huggingface wikiann [huggingface=True]
lang='ar' 
script="scripts/ner/wikiann_og.py"
predict_dataset = load_dataset(script, lang, cache_dir=CACHE_DIR)
print(predict_dataset)
print(predict_dataset['test']['tokens'][:2])
print(predict_dataset['test']['ner_tags'][:2])

Reusing dataset wikiann (.cache/wikiann/ar/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 988.45it/s]

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
})
[['تعلم', 'في', 'جامعة', 'نورث', 'وسترن', 'في', '.'], ['تحويل', 'ده\u200cشهر', '(', 'مقاطعة', 'كلاردشت', ')']]
[[0, 0, 3, 4, 4, 4, 0], [0, 5, 6, 6, 6, 6]]


In [24]:
## for test langs not present in huggingface wikiann [huggingface=False]
lang='kab' 
script="scripts/ner/wikiann_og.py"
predict_dataset = load_dataset(script, lang, cache_dir=CACHE_DIR)
print(predict_dataset)
print(predict_dataset['test']['tokens'][:2])
print(predict_dataset['test']['ner_tags'][:2])

Dataset wikiann_og downloaded and prepared to .cache/wikiann_og/kab/1.1.0/bd5069ae42633af8e332aa82fb4f866eafd1dffc876e6bae94c18978db74e5d7. Subsequent calls will reuse this data.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 792.28it/s]

DatasetDict({
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3004
    })
})
[['Aṣqif', 'n', 'Ṭmana'], ['Tizi', 'Wezzu']]
[[5, 6, 6], [5, 6]]


In [26]:
## for NORWEGIAN_NER all_lang[all_lang['dataset']=='norwegian_ner']
lang='bokmaal' 
script="scripts/ner/norwegian_ner.py"
predict_dataset = load_dataset(script, lang, cache_dir=CACHE_DIR)
print(predict_dataset)
print(predict_dataset['test']['tokens'][:2])
print(predict_dataset['test']['ner_tags'][:2])

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1174.22it/s]


Dataset norwegian_ner downloaded and prepared to .cache/norwegian_ner/bokmaal/1.0.0/d326c5669fdd52e99c78eaa6c9c8a8758a48f7494bf59ed17e12814eb349e443. Subsequent calls will reuse this data.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 516.71it/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'pos_tags', 'ner_tags'],
        num_rows: 15696
    })
    validation: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'pos_tags', 'ner_tags'],
        num_rows: 2410
    })
    test: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'pos_tags', 'ner_tags'],
        num_rows: 1939
    })
})
[['Honnørordene', 'er', '"', 'dristig', 'formspråk', '"', ',', '"', 'nyskapning', '"', 'og', '"', 'livgivende', 'kontrast', '"', '.'], ['Jeg', 'ser', 'et', 'landskap', 'som', 'er', 'såret', 'og', 'i', 'tilbaketrekning', '.']]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


### 4. dialect identification

In [85]:
counts={'train.csv':0,'dev.csv':0,'test.csv':0}
count_lang=0
arabic='data/dialect-identification/arabic/MADAR/MADAR_Corpus'
print('arabic')
count_lang+=1
for f in ['train.csv','dev.csv','test.csv']:
    print(f)
    df=pd.read_csv(os.path.join(arabic,f))
    counts[str(f)]+=len(df)
    print(dict(df['label'].value_counts()))
    print()
    
root='data/dialect-identification/'
for f in os.listdir(root):
    if f!='arabic' and not str(f).startswith('.'):
        print(f)
        count_lang+=1
        for f1 in os.listdir(os.path.join(root,f)):
            print(f1)
            df=pd.read_csv(os.path.join(root,f,f1))
            if str(f1) in ['train.csv','dev.csv','test.csv']:
                counts[str(f1)]+=len(df)
            print(dict(df['label'].value_counts()))
            print()
        print()

arabic
train.csv
{'KHA': 1600, 'RAB': 1600, 'DOH': 1600, 'DAM': 1600, 'ALX': 1600, 'AMM': 1600, 'SAL': 1600, 'JER': 1600, 'TUN': 1600, 'MUS': 1600, 'BAS': 1600, 'FES': 1600, 'TRI': 1600, 'ASW': 1600, 'MSA': 1600, 'BEI': 1600, 'RIY': 1600, 'BAG': 1600, 'BEN': 1600, 'SFX': 1600, 'ALE': 1600, 'MOS': 1600, 'CAI': 1600, 'JED': 1600, 'ALG': 1600, 'SAN': 1600}

dev.csv
{'ASW': 200, 'BAS': 200, 'JER': 200, 'AMM': 200, 'TUN': 200, 'JED': 200, 'RAB': 200, 'SFX': 200, 'MSA': 200, 'DAM': 200, 'ALG': 200, 'BEI': 200, 'TRI': 200, 'KHA': 200, 'SAL': 200, 'RIY': 200, 'ALE': 200, 'MUS': 200, 'BEN': 200, 'MOS': 200, 'BAG': 200, 'FES': 200, 'CAI': 200, 'DOH': 200, 'SAN': 200, 'ALX': 200}

test.csv
{'SAN': 200, 'JER': 200, 'BEI': 200, 'SAL': 200, 'AMM': 200, 'BAG': 200, 'RIY': 200, 'ALE': 200, 'MUS': 200, 'SFX': 200, 'ALG': 200, 'BAS': 200, 'JED': 200, 'TRI': 200, 'MOS': 200, 'ALX': 200, 'FES': 200, 'CAI': 200, 'TUN': 200, 'RAB': 200, 'MSA': 200, 'KHA': 200, 'DOH': 200, 'ASW': 200, 'DAM': 200, 'BEN': 200}

In [86]:
print(counts,count_lang)

{'train.csv': 52026, 'dev.csv': 11779, 'test.csv': 6850} 8


### 5. question-anwering

In [44]:
train_datapath='data/Question-Answering/SDQA-gold-task/sdqa-train-all.json'
dev_datapath='data/Question-Answering/SDQA-gold-task/sdqa-dev-all.json'
test_datapath='data/Question-Answering/SDQA-gold-task/sdqa-test-all.json'
language_dialect_identifier="{lang}-id-{dialect}" #look at id field, eg: "english-6037841464917965779-nga"

### 6. sentiment analysis

In [87]:
counts={'train.csv':0,'dev.csv':0,'test.csv':0}
arabic='data/sentiment_analysis/arabic'
print('arabic')
for f in os.listdir(arabic):
    print(f)
    df=pd.read_csv(os.path.join(arabic,f))
    for split in counts.keys():
        if str(f).endswith(split):
            counts[split]+=len(df)
    print(dict(df['label'].value_counts()))
    print()

arabic
aeb_Latn-train.csv
{'positive': 36942, 'negative': 30589, 'neutral': 2468}

arb_arab-train.csv
{'positive': 4797, 'neutral': 1923, 'negative': 1783, 'mixed': 324}

arq_arab-test.csv
{'negative': 7512, 'positive': 7448}

ar-lb_arab-test.csv
{5: 665, 4: 232, 3: 142, 1: 99, 2: 37}

aeb_Arab-test.csv
{'positive': 1701, 'negative': 1701}

sau_arab-train.csv
{'negative': 6080, 'positive': 1487, 'neutral': 866}

jor_arab-test.csv
{'positive': 295, 'negative': 245}

ary_arab-test.csv
{'positive': 914, 'neutral': 389, 'negative': 279, 'mixed': 58}

jor_arab-train.csv
{'negative': 655, 'positive': 605}

aeb_Latn-test.csv
{'positive': 15769, 'negative': 13178, 'neutral': 1054}

ar-lb_arab-train.csv
{5: 1648, 4: 502, 3: 276, 1: 204, 2: 111}

arz_arab-test.csv
{'objective': 2028, 'negative': 486, 'neutral': 254, 'positive': 235}

arz_arab-train.csv
{'objective': 4663, 'negative': 1198, 'neutral': 578, 'positive': 564}

aeb_Arab-train.csv
{'positive': 7155, 'negative': 6516}

arb_arab-test.cs

In [89]:
print(counts)

{'train.csv': 150664, 'dev.csv': 0, 'test.csv': 62120}


### 7. topic classification

- we train one lang per group (identifier: langgroup) and evaluate on all other

#### we only consider the following languages from the sib-200 dataset

In [90]:
import json
import pandas as pd
f = open('metadata/topic_metadata.json')
metadata = json.load(f)
# Closing file
f.close()

In [91]:
all_lang=pd.DataFrame(metadata).T
all_lang

,lang,code,langgroup
lmo_Latn,lombard,lmo_Latn,italian
eng_Latn,English,eng_Latn,English
ita_Latn,italian,ita_Latn,italian
fur_Latn,friulian,fur_Latn,italian
scn_Latn,sicilian,scn_Latn,italian
srd_Latn,sardinian,srd_Latn,italian
vec_Latn,venetian,vec_Latn,italian
azb_Arab,south,azb_Arab,azarbaijani
azj_Latn,north,azj_Latn,azarbaijani
tur_Latn,Turkish,tur_Latn,azarbaijani


In [94]:
counts={'train.csv':0,'dev.csv':0,'test.csv':0}
datapath='data/topic_class'
lang='lmo_Latn'
for lang in all_lang['code']:
    for f in os.listdir(os.path.join(datapath, lang)):
        if f!='labels.txt':
            print(f)
            df=pd.read_csv(os.path.join(datapath,lang,f))
            print(dict(df['label'].value_counts()))
            print()
            counts[str(f)]+=len(df)
            print(lang,f, len(df))

test.csv
{'science/technology': 51, 'travel': 40, 'politics': 30, 'sports': 25, 'health': 22, 'entertainment': 19, 'geography': 17}

lmo_Latn test.csv 204
dev.csv
{'science/technology': 25, 'travel': 20, 'politics': 14, 'sports': 12, 'health': 11, 'entertainment': 9, 'geography': 8}

lmo_Latn dev.csv 99
train.csv
{'science/technology': 176, 'travel': 138, 'politics': 102, 'sports': 85, 'health': 77, 'entertainment': 65, 'geography': 58}

lmo_Latn train.csv 701
test.csv
{'science/technology': 51, 'travel': 40, 'politics': 30, 'sports': 25, 'health': 22, 'entertainment': 19, 'geography': 17}

eng_Latn test.csv 204
dev.csv
{'science/technology': 25, 'travel': 20, 'politics': 14, 'sports': 12, 'health': 11, 'entertainment': 9, 'geography': 8}

eng_Latn dev.csv 99
train.csv
{'science/technology': 176, 'travel': 138, 'politics': 102, 'sports': 85, 'health': 77, 'entertainment': 65, 'geography': 58}

eng_Latn train.csv 701
test.csv
{'science/technology': 51, 'travel': 40, 'politics': 30, 'spo

{'science/technology': 176, 'travel': 138, 'politics': 102, 'sports': 85, 'health': 77, 'entertainment': 65, 'geography': 58}

yue_Hant train.csv 701
test.csv
{'science/technology': 51, 'travel': 40, 'politics': 30, 'sports': 25, 'health': 22, 'entertainment': 19, 'geography': 17}

zho_Hans test.csv 204
dev.csv
{'science/technology': 25, 'travel': 20, 'politics': 14, 'sports': 12, 'health': 11, 'entertainment': 9, 'geography': 8}

zho_Hans dev.csv 99
train.csv
{'science/technology': 176, 'travel': 138, 'politics': 102, 'sports': 85, 'health': 77, 'entertainment': 65, 'geography': 58}

zho_Hans train.csv 701
test.csv
{'science/technology': 51, 'travel': 40, 'politics': 30, 'sports': 25, 'health': 22, 'entertainment': 19, 'geography': 17}

zho_Hant test.csv 204
dev.csv
{'science/technology': 25, 'travel': 20, 'politics': 14, 'sports': 12, 'health': 11, 'entertainment': 9, 'geography': 8}

zho_Hant dev.csv 99
train.csv
{'science/technology': 176, 'travel': 138, 'politics': 102, 'sports': 

In [95]:
print(counts, len(all_lang))

{'train.csv': 27339, 'dev.csv': 3861, 'test.csv': 7956} 39


### 8. reading comprehension

- we only consider the following evaluation languages from Belebele dataset

In [51]:
f = open('metadata/rcmc_metadata.json')
metadata = json.load(f)
# Closing file
f.close()

In [52]:
all_lang=pd.DataFrame(metadata).T
all_lang.head(10)

,lang,code,langgroup
zho_Hans,Chinese (Simplified),zho_Hans,chinese
zho_Hant,Chinese (Traditional),zho_Hant,chinese
eng_Latn,English,eng_Latn,English
nso_Latn,Northern Sotho,nso_Latn,sotho
sot_Latn,Southern Sotho,sot_Latn,sotho
acm_Arab,Mesopotamian Arabic,acm_Arab,arabic
apc_Arab,North Levantine Arabic,apc_Arab,arabic
arb_Arab,MSA (Arabic),arb_Arab,arabic
ars_Arab,Najdi Arabic,ars_Arab,arabic
ary_Arab,Moroccan Arabic,ary_Arab,arabic


In [54]:
datapath='datapath/reading-comprehension/Belebele'
sample_test_file='arb_Arab.jsonl'
first_example={"link":"https:\/\/en.wikibooks.org\/wiki\/Accordion\/Right_hand","question_number":1,"flores_passage":"Make sure your hand is as relaxed as possible while still hitting all the notes correctly - also try not to make much extraneous motion with your fingers. This way, you will tire yourself out as little as possible. Remember there's no need to hit the keys with a lot of force for extra volume like on the piano. On the accordion, to get extra volume, you use the bellows with more pressure or speed.","question":"According to the passage, what would not be considered an accurate tip for successfully playing the accordion?","mc_answer1":"For additional volume, increase the force with which you hit the keys","mc_answer2":"Keep unnecessary movement to a minimum in order to preserve your stamina","mc_answer3":"Be mindful of hitting the notes while maintaining a relaxed hand","mc_answer4":"Increase the speed with which you operate the bellows to achieve extra volume","correct_answer_num":"1","dialect":"eng_Latn","ds":"2023-05-03"}
print(first_example)

{'link': 'https:\\/\\/en.wikibooks.org\\/wiki\\/Accordion\\/Right_hand', 'question_number': 1, 'flores_passage': "Make sure your hand is as relaxed as possible while still hitting all the notes correctly - also try not to make much extraneous motion with your fingers. This way, you will tire yourself out as little as possible. Remember there's no need to hit the keys with a lot of force for extra volume like on the piano. On the accordion, to get extra volume, you use the bellows with more pressure or speed.", 'question': 'According to the passage, what would not be considered an accurate tip for successfully playing the accordion?', 'mc_answer1': 'For additional volume, increase the force with which you hit the keys', 'mc_answer2': 'Keep unnecessary movement to a minimum in order to preserve your stamina', 'mc_answer3': 'Be mindful of hitting the notes while maintaining a relaxed hand', 'mc_answer4': 'Increase the speed with which you operate the bellows to achieve extra volume', 'cor

In [55]:
combined_train_file=datapath='datapath/reading-comprehension/Belebele/train.jsonl'

### 9. Natural language Inference

- we perform cross-lingual transfer on translate-test langs from eng_Latn

In [70]:
f = open('metadata/nli_metadata.json')
metadata = json.load(f)
# Closing file
f.close()

In [71]:
all_lang=pd.DataFrame(metadata).T
all_lang.head(10)

,lang,code,langgroup
lmo_Latn,lombard,lmo_Latn,italian
eng_Latn,English,eng_Latn,English
ita_Latn,italian,ita_Latn,italian
fur_Latn,friulian,fur_Latn,italian
scn_Latn,sicilian,scn_Latn,italian
srd_Latn,sardinian,srd_Latn,italian
vec_Latn,venetian,vec_Latn,italian
azb_Arab,south,azb_Arab,azarbaijani
azj_Latn,north,azj_Latn,azarbaijani
tur_Latn,Turkish,tur_Latn,azarbaijani


In [59]:
all_eval_langs=["eng_Latn","lmo_Latn","ita_Latn","fur_Latn","scn_Latn","srd_Latn","vec_Latn","azb_Arab","azj_Latn","tur_Latn","kmr_Latn","ckb_Arab","nno_Latn","nob_Latn","lim_Latn","ltz_Latn","nld_Latn","lvs_Latn","ltg_Latn","acm_Arab","acq_Arab","aeb_Arab","ajp_Arab","apc_Arab","arb_Arab","ars_Arab","ary_Arab","arz_Arab","kab_Latn","asm_Beng","ben_Beng","lij_Latn","oci_Latn","yue_Hant","zho_Hans","zho_Hant","glg_Latn","spa_Latn","por_Latn","nso_Latn","sot_Latn"]

In [60]:
print(all_eval_langs)

['eng_Latn', 'lmo_Latn', 'ita_Latn', 'fur_Latn', 'scn_Latn', 'srd_Latn', 'vec_Latn', 'azb_Arab', 'azj_Latn', 'tur_Latn', 'kmr_Latn', 'ckb_Arab', 'nno_Latn', 'nob_Latn', 'lim_Latn', 'ltz_Latn', 'nld_Latn', 'lvs_Latn', 'ltg_Latn', 'acm_Arab', 'acq_Arab', 'aeb_Arab', 'ajp_Arab', 'apc_Arab', 'arb_Arab', 'ars_Arab', 'ary_Arab', 'arz_Arab', 'kab_Latn', 'asm_Beng', 'ben_Beng', 'lij_Latn', 'oci_Latn', 'yue_Hant', 'zho_Hans', 'zho_Hant', 'glg_Latn', 'spa_Latn', 'por_Latn', 'nso_Latn', 'sot_Latn']


In [61]:
train_lang='eng_Latn'

In [67]:
lang=train_lang
dataset = load_dataset("scripts/nli/dialect_nli.py", lang,
            cache_dir=CACHE_DIR)
print(dataset)
print(dataset['train'].features)

Reusing dataset dialect_nli (.cache/dialect_nli/eng_Latn/1.1.0/b69815628a902151a9f2b158e6be8fabf359868aa4a25c29c09ff689455041b9)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 363.58it/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 392702
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 5010
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 2490
    })
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=3, names=['entailment', 'neutral', 'contradiction'], id=None)}


In [69]:
lang='lmo_Latn'
dataset = load_dataset("scripts/nli/dialect_nli.py", lang,
            cache_dir=CACHE_DIR)
print(dataset)
print(dataset['test'].features)

Reusing dataset dialect_nli (.cache/dialect_nli/lmo_Latn/1.1.0/b69815628a902151a9f2b158e6be8fabf359868aa4a25c29c09ff689455041b9)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 963.69it/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 0
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 5010
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 0
    })
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=3, names=['entailment', 'neutral', 'contradiction'], id=None)}


### 10. Machine Translation